### PyNAMod
1) Получение огрубленных моделей структур ДНК, генерация линейных ДНК, объединение моделей.
2) Исследование конформаций с помощью моделирования методом Монте Карло.

![alt text](params.png "Title")

### Установка
https://github.com/intbio/PyNAMod \
Создание окружения:\
conda env create --file environment.yml

### Анализ полноатомоных моделей

In [27]:
import pynamod
import torch

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Инициализация с помощью объекта mda Universe, pdb файла или pdb id
nucl = pynamod.CG_Structure(pdb_id='3LZ0')

Sending GET request to https://files.rcsb.org/download/3LZ0.pdb to fetch 3LZ0's pdb file as a string.


Получение геометрических параметров

In [8]:
nucl.analyze_dna(leading_strands=['I'])

Получение огрубленной модели белка

In [9]:
nucl.analyze_protein()

Визуализация

In [6]:
# Перемещение первой пары в начало координат
nucl.dna.move_to_coord_center()

nucl.view_structure()

NGLWidget()

### Генерация линейной ДНК

In [10]:
seq = 'atcg'*2
dna_gen = pynamod.CG_Structure()
dna_gen.build_dna(seq)

In [11]:
dna_gen.view_structure()


NGLWidget()

### Объединение структур

Инициализация стурктуры с белком cas9

In [4]:
cas = pynamod.CG_Structure(pdb_id='5Y36')
cas.analyze_dna(leading_strands=['C'],sel='segid C D')

Sending GET request to https://files.rcsb.org/download/5Y36.pdb to fetch 5Y36's pdb file as a string.


In [5]:
cas.analyze_protein()

In [6]:
cas.view_structure()

NGLWidget()

Объединение с нуклеосомой и линейной ДНК

In [13]:
nucl.append_structures([dna_gen,cas])

In [14]:
nucl.view_structure()

NGLWidget()

### Моделирование методом Монте-Карло

Подготовка расчетов энергий

In [58]:
import torch
import numpy as np

In [16]:
en = pynamod.Energy()
en.set_energy_matrices(nucl)

Полная энергия: энергия изгиба ДНК + энергия электростатических взаимодействий + энергия взаимодействия частиц (в виде потенциала Леннарда Джонса или сигмоиды)

In [17]:
en.get_full_energy(nucl)

mod_Tensor([53744.3281])

Генерация списка шагов, которые будут изменяться при моделирование

In [21]:
movable_steps = torch.ones(nucl.dna.step_params.shape[0],dtype=bool)
movable_steps[0] = False

In [67]:
movable_steps

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True])

Запуск моделирования

In [22]:
it = pynamod.Iterator(nucl,en)

In [25]:
it.run(movable_steps,target_accepted_steps=500,max_steps=10000)

Steps:   0%|          | 0/10000 [00:00<?, ?it/s]

Acceptance rate:   0%|          

target accepted steps reached
accepted_steps 500


In [26]:
nucl.view_structure()

NGLWidget()

### Применение ограничений на взаимное положение пар ДНК

Для получения определенных конформаций структуры в расчет можно вводить ожидаемые геометрические параметры между любыми двумя парами ДНК

In [20]:
from pynamod.external_forces.restraint import Restraint
from pynamod.energy.energy_constants import BDNA_step
from pynamod.geometry.bp_step_geometry import rebuild_by_full_par_frame_numba

Получение кольцевой ДНК из линейной

In [71]:
# Индексы пар, на которые накладывается ограничение
ind1,ind2 = 0, -1

# Коэффицент этого ограничения в формуле полной энергии
force_const = 500

# Целевые геометрические параметры
params = torch.Tensor(BDNA_step[6:])

# Допустимое отклонение
dev = torch.ones(1)*0.1

# Инициализация
restr = Restraint(ind1,ind2,force_const,params,dev)

Подготовка расчета энергии

In [72]:
en_linear = pynamod.Energy()
dna_gen.move_to_torch()
en_linear.set_energy_matrices(dna_gen)
en_linear.add_restraints([restr])

In [49]:
it = pynamod.Iterator(dna_gen,en_linear)
movable_steps = torch.ones(dna_gen.steps_params.shape[0],dtype=bool)

In [50]:
frames,energies = it.run(movable_steps=movable_steps,target_accepted_steps=100000)

Steps:   0%|          | 0/1000000.0 [00:00<?, ?it/s]

Acceptance rate:   0%|          

interrupted!
accepted_steps 27295


In [51]:
dna_gen.base_ref_frames = rebuild_by_full_par_frame_numba(np.array(frames[-1]))

In [52]:
dna_gen.view_structure()

NGLWidget()

In [31]:
nucl.dna.ref_frames[10].numpy()

array([[ 0.98677093, -0.13164997, -0.09461191],
       [ 0.14793247,  0.96993309,  0.19325058],
       [ 0.06632579, -0.20469023,  0.97657708]])

In [32]:
from scipy.spatial.transform import Rotation

In [34]:
Rotation.from_matrix(nucl.dna.ref_frames[10].numpy()).as_quat()

array([-0.10032542, -0.04057423,  0.07048593,  0.99162507])

In [35]:
Rotation.from_matrix(nucl.dna.ref_frames[10].numpy()).as_rotvec()

array([-0.20121287, -0.08137576,  0.14136672])

In [38]:
import numpy as np

In [36]:
def quat_mult_old(q1,q2):
    prod = np.zeros(q1.shape)
    prod[:,0] = q2[:,0]*q1[:,0] - np.sum(q1[:,1:]*q2[:,1:],axis=1)
    prod[:,1:] = q2[:,0].reshape(-1,1)*q1[:,1:] + q1[:,0].reshape(-1,1)*q2[:,1:] + np.cross(q1[:,1:],q2[:,1:])
    return prod


def rotate_old(quat,v):
    u = quat[:,1:]
    rot = 2*np.sum(u*v,axis=1).reshape(-1,1)*u 
    rot += (quat[:,0]**2 - np.linalg.norm(u,axis=1)**2).reshape(-1,1)*v
    rot += 2*quat[:,0].reshape(-1,1)*np.cross(u,v)
    return rot

def rotate(quat,v):
    conj = get_conj(quat)
    q_v = np.hstack([np.zeros((quat.shape[0],1)),v])
    prod = quat_mult(quat,q_v)
    return quat_mult(prod,conj)[:,1:]

def get_quat(phi,vec):
    phi = phi.reshape(-1,1)
    vec = np.tile(vec,phi.shape[0]).reshape(-1,3)
    quat = np.hstack([np.cos(phi/2),vec*np.sin(phi/2)])
    return quat

def get_conj(quat):
    return quat*[1,-1,-1,-1]

def normalize(quat):
    sin = np.sin(np.arccos(quat[:,0])).reshape(-1,1)
    vec = quat[:,1:]/sin
    quat[:,1:] /= np.linalg.norm(vec,axis=1).reshape(-1,1)
    return quat   

def quat_from_euler(a,b,g):
    cosa, sina = np.cos(a),np.sin(a)
    cosb, sinb = np.cos(b),np.sin(b)
    cosg, sing = np.cos(g),np.sin(g)
    q = np.zeros((cosa.shape[0],4))
    q[:,3] = cosb*cosa*sing + cosb*sina*cosg
    q[:,2] = sinb*cosa*cosg + sinb*sina*sing
    q[:,1] =  - sinb*cosa*sing + sinb*sina*cosg
    q[:,0] =  cosb*cosa*cosg - cosb*sina*sing
    return q

def rebuild_base_ref_frames_quat(steps_frames,bp_frames):
    steps_frames = np.copy(steps_frames)

    steps_frames[1:,3:] = np.deg2rad(steps_frames[1:,3:])
    z = np.array((0,0,1))
    y = np.array((0,1,0))


    gamma = np.linalg.norm(steps_frames[1:,3:5],axis=1)
    cos_phi =  steps_frames[1:,4]/gamma
    phi = np.arccos(cos_phi)
    phi[steps_frames[1:,3]<0] *= -1
    q1 = get_quat(steps_frames[1:,5]/2 - phi,z)
    qm = quat_mult(q1,get_quat(gamma/2,y))
    qm = quat_mult(qm,get_quat(phi,z))
    
    q2 = quat_mult(q1,get_quat(gamma,y))
    q2 = quat_mult(q2,get_quat(steps_frames[1:,5]/2 + phi,z))
    o2 = rotate(qm,steps_frames[1:,:3])
    bp_frames[0,:4] = [0,0,0,1]
    for i,q in enumerate(q2):
        bp_frames[i+1,:4] = quat_mult(bp_frames[i,:4].reshape(-1,4),q.reshape(-1,4))
    bp_frames[1:,4:] = np.cumsum(rotate(bp_frames[:-1,:4],o2),axis=1)
    
    return bp_frames

def quat_mult(a,b):
    res = np.zeros(a.shape)
    res[:,0] = a[:,0] * b[:,0] - a[:,1] * b[:,1] - a[:,2] * b[:,2] - a[:,3] * b[:,3]
    res[:,1] = a[:,0] * b[:,1] + a[:,1] * b[:,0] + a[:,2] * b[:,3] - a[:,3] * b[:,2]
    res[:,2] = a[:,0] * b[:,2] - a[:,1] * b[:,3] + a[:,2] * b[:,0] + a[:,3] * b[:,1]
    res[:,3] = a[:,0] * b[:,3] + a[:,1] * b[:,2] - a[:,2] * b[:,1] + a[:,3] * b[:,0]
    
    return(res)

In [41]:
q = get_quat(np.array((np.pi)),np.ones(3))

In [51]:
rotate_old(q,np.array((1,0,0)))

array([[-1.,  2.,  2.]])

In [46]:
np.array(((1,0,0),)).shape

(1, 3)

In [47]:
q.shape

(1, 4)